In [1]:
!pip install --upgrade --force-reinstall numpy scipy scikit-learn

  Using cached numpy-2.2.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
  Using cached scipy-1.15.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
  Using cached scikit_learn-1.6.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (18 kB)
  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
Using cached numpy-2.2.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.4 MB)
Using cached scipy-1.15.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (37.6 MB)
Using cached scikit_learn-1.6.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (13.5 MB)
Using cached joblib-1.4.2-py3-none-any.whl (301 kB)
Using cached threadpoolctl-3.6.0-py3-none-any.whl (18 kB)
  Attempting uninstall: threadpoolctl
    Found existing installation: threadpoolctl 3.6.0
    Uninstalling threadpoolctl-3.6.0:
      Successfully uni

In [2]:
pip install gradio

In [9]:
import joblib
# Load the model
model=joblib.load('/content/stacking_clas.pkl')
model

MultiOutputClassifier(estimator=StackingClassifier(estimators=[('RF',
                                                                RandomForestClassifier()),
                                                               ('DT',
                                                                DecisionTreeClassifier()),
                                                               ('xgb_model',
                                                                XGBClassifier(base_score=None,
                                                                              booster=None,
                                                                              callbacks=None,
                                                                              colsample_bylevel=None,
                                                                              colsample_bynode=None,
                                                                              colsample_bytree=None,
                                                                              device=None,
                                                                              early_stopping_rounds=None,
                                                                              enable_categorical=False,
                                                                              eval_metric=None,
                                                                              feat...
                                                                              interaction_constraints=None,
                                                                              learning_rate=None,
                                                                              max_bin=None,
                                                                              max_cat_threshold=None,
                                                                              max_cat_to_onehot=None,
                                                                              max_delta_step=None,
                                                                              max_depth=None,
                                                                              max_leaves=None,
                                                                              min_child_weight=None,
                                                                              missing=nan,
                                                                              monotone_constraints=None,
                                                                              multi_strategy=None,
                                                                              n_estimators=None,
                                                                              n_jobs=None,
                                                                              num_parallel_tree=None,
                                                                              random_state=None, ...))],
                                                   final_estimator=DecisionTreeClassifier()))

In [10]:
import numpy as np

def predict_kubernetes_issue(*features):
    features = np.array(features).reshape(1, -1)
    prediction = model.predict(features)[0]  # Get the predicted array
    # Define issue descriptions for 0 and 1 values
    issue_descriptions = [
        ("A failure in a Kubernetes node or pod occurred.", "There is no issue in Kubernetes node or pod."),
        ("Resource exhaustion detected (CPU, Memory, Disk).", "There is no resource exhaustion."),
        ("Network or connectivity issues detected.", "There are no network or connectivity issues."),
        ("Service disruptions based on logs and events detected.", "There are no service disruptions.")
    ]

    # Generate a detailed explanation for each issue
    result_lines = [
        f"{i+1}. {issue_descriptions[i][1 - prediction[i]]}"  # Picks the correct text based on 0 or 1
        for i in range(len(issue_descriptions))
    ]

    # Format the final output
    result_text = "Predicted Kubernetes Issues:\n" + "\n".join(result_lines)

    return result_text  # Ensure function returns text instead of array

In [11]:
import gradio as gr
input_components = [
            gr.Number(label='CPU Usage %'),
            gr.Number(label='Memory Usage %'),
            gr.Number(label='Disk Usage %'),
            gr.Number(label='Network Latency (ms)'),
            gr.Number(label='Request Rate (req/s)'),
            gr.Number(label='Pod Restarts'),
            gr.Number(label='Error Rate'),
            gr.Number(label='Node Temperature'),
            gr.Number(label='Connection Attempts'),
            gr.Number(label='Queue Length'),
            gr.Number(label='Log Messages'),
            gr.Number(label='Event Messages')]
iface = gr.Interface(fn=predict_kubernetes_issue, inputs=input_components, outputs="text")

In [12]:
iface.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://46f03403bd26a85767.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
